In [2]:
import torch
import torch.nn as nn

In [3]:
class NGLCM(nn.Module):
    def __init__(self, dim, colors=16, offset=1):
        """NGLCM layer implementation
        
        Parameters:
            dim : int
                Dimensions of image (dim * dim)
            colors : int
                Size of colors space
            
        """
        
        super(NGLCM, self).__init__()
    
        self.divide = 256 // 16
        
        self.a = nn.Parameter(torch.zeros([colors, 1]))
        nn.init.xavier_uniform_(self.a)
        
        self.b = nn.Parameter(torch.zeros([colors, 1]))
        nn.init.xavier_uniform_(self.b)
        
        self.cooc = torch.zeros([dim*dim, dim*dim])
        for i in range(dim*dim):
            self.cooc[i, i] = 1
            if i + offset >= 0 and i + offset < dim*dim:
                self.cooc[i, i + offset] = -1
        
    def forward(self, x):
        a = x.view(1, -1)
        b = a.mm(self.cooc)
        
        return torch.clamp(a - self.a, 0, 1).mm(torch.clamp(b - self.b, 0, 1).t())

In [27]:
nglcm = NGLCM(224)
from lib import datasets

data = datasets.datasets()
batch_loader = data.create_dataset(batch_size=1, img_size=224, data_aug=True)

img = None
label = None

for a, b in (batch_loader['train']):
    # a - batch_size x channels x img_size x img_size - data
    # b - batch_size - labels
    img = a
    label = b
    break
print(img.shape)


RuntimeError: $ Torch: not enough memory: you tried to allocate 9GB. Buy new RAM! at /pytorch/aten/src/TH/THGeneral.cpp:201

In [25]:
nglcm.forward(img).shape

torch.Size([16, 16])